### Reading Data

In [0]:
events =spark.table('ecommerce.silver.events')

In [0]:
events.columns

###  Descriptive stats

In [0]:
events.select('price').describe().show()

##### median

In [0]:
events.approxQuantile('price',[0.50],0.01)

#### Quatiles

In [0]:
events.approxQuantile('price',[0.25,0.50,0.75],0.01)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
weekday=events.withColumn(
    "isweekday",
    ~dayofweek(col("event_date")).isin([1,7])
    )
weekday.groupBy("isweekday","event_type").count().orderBy("event_type").show()

### # Correlation

In [0]:
events.stat.corr('price','category_id')

In [0]:

# Feature engineering
features = events.withColumn("hour", hour("event_time")) \
    .withColumn("day_of_week", dayofweek("event_date")) \
    .withColumn("price_log", log(col("price")+1)) \
    .withColumn("time_since_first_view",
    unix_timestamp(col("event_time")) -
    unix_timestamp(first(col("event_time")).over(Window.partitionBy("user_id").orderBy("event_time")))
    
    )

In [0]:

display(features)    